In [1]:
!wget https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip

--2021-08-31 12:52:28--  https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6022:18::a27d:4212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip [following]
--2021-08-31 12:52:28--  https://www.dropbox.com/s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc8bf83a7712f889be23ea3d72f.dl.dropboxusercontent.com/cd/0/inline/BVRWrj5IQ5ni7-Pfaix4EQjT-kzXBXJBhZsxiq42QZcsF--tSSpVRwU3kDUu8URLy_uNsdhn2UYCgkobDmXEGwDwxUX8nADLbYuNliXod8zfu-KjPnJCq_mblEeaMJSwvgDhHQQLqqy4jlOLqkYk6INs/file# [following]
--2021-08-31 12:52:29--  https://ucc8bf83a7712f889be23ea3d72f.dl.dropboxusercontent.com/cd/0/inline/BVRWrj5IQ5ni7-Pfaix4EQjT-kzXBXJBhZsxiq42QZcsF--tSS

In [2]:
# Let's unzip the file
!unzip -q "/content/movie-reviews-dataset.zip"

In [3]:
from tensorflow.keras.preprocessing import  text_dataset_from_directory
from tensorflow.strings import regex_replace
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, GRU, Embedding, Dropout, Bidirectional

In [4]:
def prepareData(dir):
  data = text_dataset_from_directory(dir)
  return data.map(
      lambda text, label: (regex_replace(text, "<br />", " "), label),
  )

In [5]:
train_data = prepareData("/content/movie-reviews-dataset/train")
test_data = prepareData("/content/movie-reviews-dataset/test")

for text_batch, label_batch in train_data.take(1):
  print(text_batch.numpy()[0])
  print(label_batch.numpy()[0])

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
b'Somehow they summed up the 60\'s, ten years that radically changed our country, in four hours. And what a painful four hours it was. They trivilized the major events and happenings and they "claimed" it was about two families yet you barely saw the african-american family. If I were NBC I would be ashamed and embarrassed for airing such trash. What was amusing was this happy-go-lucky family you saw in the very beginning was tortured in so many ways, but managed to attend every major 60\'s event through the country. And the second family was such a non-factor. They devoted maybe five or six scenes total to this family. That poor son... Please NBC, do not make any movies about any other eras....leave that to PBS and the History Channel'
0


In [6]:
model =Sequential()

In [7]:
model.add(Input(shape=(1,), dtype="string"))

In [8]:
max_tokens= 1000
max_len = 100
vectorize_layer = TextVectorization(
    max_tokens = max_tokens,
    output_mode = 'int',
    output_sequence_length = max_len,
)

In [9]:
train_texts = train_data.map(lambda text, label: text)
vectorize_layer.adapt(train_texts)

model.add(vectorize_layer)

In [10]:
model.add(Embedding(max_tokens + 1, 128))

model.add(Bidirectional(GRU(64)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [11]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
model.fit(train_data, epochs=10)

Epoch 1/10
782/782 [==============================] - 104s 127ms/step - loss: 0.5367 - accuracy: 0.7168
Epoch 2/10
782/782 [==============================] - 99s 127ms/step - loss: 0.4239 - accuracy: 0.8009
Epoch 3/10
782/782 [==============================] - 100s 127ms/step - loss: 0.3887 - accuracy: 0.8216
Epoch 4/10
782/782 [==============================] - 103s 131ms/step - loss: 0.3564 - accuracy: 0.8392
Epoch 5/10
782/782 [==============================] - 101s 128ms/step - loss: 0.3258 - accuracy: 0.8554
Epoch 6/10
782/782 [==============================] - 103s 131ms/step - loss: 0.2985 - accuracy: 0.8667
Epoch 7/10
782/782 [==============================] - 102s 130ms/step - loss: 0.2699 - accuracy: 0.8810
Epoch 8/10
782/782 [==============================] - 101s 129ms/step - loss: 0.2373 - accuracy: 0.8985
Epoch 9/10
782/782 [==============================] - 105s 134ms/step - loss: 0.2086 - accuracy: 0.9119
Epoch 10/10
600/782 [======================>.......] - ETA: 23s -

In [13]:
model.evaluate(test_data)

782/782 [==============================] - 27s 34ms/step - loss: 0.7904 - accuracy: 0.7730


[0.7903835773468018, 0.7730399966239929]

In [14]:
text = "This movis is bad"

In [15]:
model.predict([text]) # 0 = Negative review, 1 = positive review

array([[0.01893267]], dtype=float32)

In [16]:
text2 = "This movis is good"

In [17]:
model.predict([text2]) # 0 = Negative review, 1 = positive review

array([[0.897671]], dtype=float32)